In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/sample_code.ipynb
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/submission_baseline.csv
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/lab/t2_wall.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/lab/t2_goal.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/lab/t2_drone.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/lab/t2_glue.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/lab/t2_plasma.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/lab/t2_floor.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/desert/t1_goal.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/assets/desert/t1_rover.png
/kaggle/input/the-blind-flight-synapse-drive-ps-1/Syn

In [2]:
import os

root = "/kaggle/input/the-blind-flight-synapse-drive-ps-1"
print("Level 1:", os.listdir(root))

for f in os.listdir(root):
    p = os.path.join(root, f)
    if os.path.isdir(p):
        print("\nInside:", f)
        print(os.listdir(p)[:20])


Level 1: ['SynapseDrive_Dataset']

Inside: SynapseDrive_Dataset
['assets', 'sample_code.ipynb', 'submission_baseline.csv', 'test', 'train']


In [3]:
base = "/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset"

print("Base contents:", os.listdir(base))

print("Train:", os.listdir(base + "/train")[:10])
print("Test:", os.listdir(base + "/test")[:10])

print("Train images sample:", os.listdir(base + "/train/images")[:5])
print("Test images sample:", os.listdir(base + "/test/images")[:5])


Base contents: ['assets', 'sample_code.ipynb', 'submission_baseline.csv', 'test', 'train']
Train: ['labels', 'images', 'velocities']
Test: ['images', 'velocities']
Train images sample: ['0006.png', '0003.png', '0013.png', '0017.png', '0007.png']
Test images sample: ['9273.png', '9292.png', '0566.png', '4353.png', '7968.png']


In [4]:
BASE = "/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset"

print("Base:", os.listdir(BASE))
print("Train:", os.listdir(BASE + "/train")[:5])
print("Test:", os.listdir(BASE + "/test")[:5])


Base: ['assets', 'sample_code.ipynb', 'submission_baseline.csv', 'test', 'train']
Train: ['labels', 'images', 'velocities']
Test: ['images', 'velocities']


In [5]:
import os
import cv2
import json
import numpy as np
import pandas as pd
from heapq import heappush, heappop


In [6]:
BASE_COST = {
    "lab": {0:1.0, 2:3.0, 3:1.0, 4:2.0},
    "forest": {0:1.5, 2:2.8, 3:1.5, 4:2.5},
    "desert": {0:1.2, 2:3.7, 3:1.2, 4:2.2},
}


In [7]:
def detect_terrain(image):
    mean_val = image.mean()
    if mean_val > 180:
        return "desert"
    elif mean_val < 100:
        return "lab"
    return "forest"


In [8]:
def dummy_grid():
    grid = np.zeros((20,20), dtype=int)

    grid[0,:] = 1
    grid[-1,:] = 1
    grid[:,0] = 1
    grid[:,-1] = 1
    
    grid[1,1] = 3
    grid[18,18] = 4
    
    return grid


In [9]:
DIRS = [(0,1,'r'),(0,-1,'l'),(1,0,'d'),(-1,0,'u')]

def dijkstra(grid, boosts, terrain):
    n = 20
    start = None
    goal = None
    
    for i in range(n):
        for j in range(n):
            if grid[i][j] == 3:
                start = (i,j)
            if grid[i][j] == 4:
                goal = (i,j)
                
    pq = []
    heappush(pq,(0,start,""))
    visited = set()
    
    while pq:
        cost,(x,y),path = heappop(pq)
        if (x,y) in visited:
            continue
        visited.add((x,y))
        
        if (x,y) == goal:
            return path
        
        for dx,dy,ch in DIRS:
            nx,ny = x+dx,y+dy
            if 0<=nx<20 and 0<=ny<20:
                cell = grid[nx][ny]
                if cell == 1:
                    continue
                    
                base = BASE_COST[terrain].get(cell, 5.0)
                step_cost = base - boosts[nx][ny]
                
                heappush(pq,(cost+step_cost,(nx,ny),path+ch))
    
    return ""


In [10]:
results = []

test_images = sorted(os.listdir(BASE + "/test/images"))

for file in test_images:
    img_id = file.replace(".png","")
    
    img = cv2.imread(BASE + "/test/images/" + file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    boosts = json.load(open(BASE + "/test/velocities/" + img_id + ".json"))["boost"]
    boosts = np.array(boosts)
    
    terrain = detect_terrain(img)
    
    grid = dummy_grid()
    
    path = dijkstra(grid, boosts, terrain)
    
    results.append([img_id, path])

df = pd.DataFrame(results, columns=["image_id","path"])
df.to_csv("submission.csv", index=False)

df.head()


,image_id,path
0,0001,dddddrdrdrdrdrdrdrdrdrdrdrdrdrrrrr
1,0002,rrrrrrdrdrdrdrdrdrdrdrdrdrdrdddddd
2,0003,ddddddddddddddddrdrrrrrrrrrrrrrrrr
3,0004,rrrrrrrrrrrrrrrrdrdddddddddddddddd
4,0005,rrrrrrrrrrrrrrrrrddddddddddddddddd


In [11]:
output_path = "/kaggle/working/submission.csv"
df.to_csv(output_path, index=False)
print("Saved to:", output_path)


Saved to: /kaggle/working/submission.csv


In [12]:

print(os.listdir("/kaggle/working"))


['submission.csv', '__notebook__.ipynb']


In [13]:
from IPython.display import FileLink
FileLink('/kaggle/working/submission.csv')


/kaggle/working/submission.csv